In [1]:
import time
import torch
from torch.profiler import profile, record_function, ProfilerActivity
import pandas as pd
from torch import nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader,TensorDataset
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
import random
import optuna
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score, confusion_matrix
from sklearn.model_selection import StratifiedKFold, train_test_split
import seaborn as sns
from collections import OrderedDict
import warnings
warnings.filterwarnings("ignore")

In [2]:
# # [测试]前向传播过程是否正常
# import torch
# import torch.nn as nn
# 
# # 假设有 3 个类别
# num_classes = 3
# batch_size = 5
# 
# # 创建一个简单的模型
# model = nn.Sequential(
#     nn.Linear(10, num_classes)
# )
# 
# # 定义损失函数
# loss_fn = nn.CrossEntropyLoss()
# 
# # 随机生成一些输入和标签
# inputs = torch.randn(batch_size, 10)
# targets = torch.randint(0, num_classes, (batch_size,))
# 
# # 前向传播
# outputs = model(inputs)
# 
# # 计算损失
# loss = loss_fn(outputs, targets)
# 
# print(loss)


In [3]:
data = pd.read_csv('Data_encoded\LSTM_data\combined_data_processed.csv')
data.head()

,duration,protocol_type_icmp,protocol_type_tcp,protocol_type_udp,service_IRC,service_X11,service_Z39_50,service_aol,service_auth,service_bgp,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,Class
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.098039,0.17,0.03,0.17,0.00,0.00,0.00,0.05,0.00,Normal
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.003922,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,Normal
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.101961,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,Dos
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,Normal
4,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.000000,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Normal


In [4]:
# 自定义数据集类
class CustomDataset(Dataset):
    # 类的构造函数。它接受两个参数features和labels，分别表示数据集的特征和标签。
    # 在初始化过程中，将这些特征和标签存储在类的实例变量中。
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    # 这是一个特殊方法，用于返回数据集的长度（即数据样本的数量）。
    # 在这个方法中，它返回了存储在features中的样本数量，即数据集的长度。
    def __len__(self):
        return len(self.features)

    # 这也是一个特殊方法，用于根据给定索引idx来获取数据集中的样本。
    # 在这个方法中，它根据索引idx从features和labels中获取对应索引的特征和标签，并将它们作为元组返回。
    def __getitem__(self, idx):
        # return self.features[idx], self.labels[idx]
        feature = self.features[idx]
        label = self.labels[idx]

        # 确保 feature 是一个数值型数组
        if isinstance(feature, np.ndarray):
            if feature.dtype.type is np.str_ or feature.dtype.type is np.object_:
                raise ValueError("Features must be numeric")

        # 如果 feature 不是一个 ndarray，或者它的 dtype 不是浮点数，尝试将其转换
        if not isinstance(feature, np.ndarray) or feature.dtype != 'float32':
            feature = np.array(feature, dtype=np.float32)

        # 转换为 PyTorch 张量
        feature = torch.tensor(feature, dtype=torch.float32)

        # 如果标签不是一个张量，转换它
        if not torch.is_tensor(label):
            label = torch.tensor(label, dtype=torch.long)

        return feature, label

In [5]:
# 定义BiLSTMLayer层模型
class BiLSTMLayer(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1):
        super(BiLSTMLayer, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)

    def forward(self, x):
        # LSTM的输出包括所有隐藏状态、最后的隐藏状态和最后的细胞状态
        output, _ = self.lstm(x)
        # 只返回输出张量，不返回隐藏状态和细胞状态
        return output[:, -1, :]  # 只返回最后一个时间步的输出

In [6]:
# 定义网络结构
class CNNBiLSTMModel(nn.Module):
    def __init__(self):
        super(CNNBiLSTMModel, self).__init__()
        self.conv1d = nn.Conv1d(in_channels=1, out_channels=64, kernel_size=122, padding='same')  # 保持输出尺寸不变
        self.maxpool1 = nn.MaxPool1d(kernel_size=5)
        self.batchnorm1 = nn.BatchNorm1d(64)
        # out.shape=(batch=32, channel = 64, seq=24(122池化后的数字))

        # input_dim = 就是nn.LSTM(input_size(x的特征维度),hidden_size,...)中的input_size,
        # 在该数据中,input_size恒为1

        self.bilstm1 = BiLSTMLayer(input_dim=64, hidden_dim=64)  # hidden_size即为上一层的输出channel

        # 此处需要将(128, ) reshape为(1,128), 因为要沿着128的方向做池化,
        # 为啥要沿128的方向,个人理解128为预测出来的特征,故继续提取特征
        self.maxpool1d2 = nn.MaxPool1d(kernel_size=5)
        self.batchnorm2 = nn.BatchNorm1d(1)

        # 第二个BiLSTM
        # input=(input_size=1, hidden_size=128, 其他默认) ,seq=25(根据上一层的输出判断的)
        self.bilstm2 = BiLSTMLayer(input_dim=1, hidden_dim=128)  # BiLSTM只取了最后一个时间步的输出
        # out.shape = (batch=32, 1(啥意思暂不明白), 256(就是128池化后的数字))

        self.dropout = nn.Dropout(0.5)  # 将上一层随机丢弃一半传入下层
        self.fc = nn.Linear(256, 5)
        # self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.conv1d(x)
        x = F.relu(x)
        x = self.maxpool1(x)
        x = self.batchnorm1(x)
        # shape=(32, 64, 24)

        x = x.permute(0, 2, 1)  # 重排维度以适配LSTM输入
        # shape=(32, 24, 64)

        # 第一个BiLSTM
        # BiLSTM.output.shape = (batch, seq, hidden_size*2) = (32, 24, 128)
        x = self.bilstm1(x)  # 但此处只取了最后一个seq, 此时x.shape=(32,128)
        x = x.unsqueeze(1)  # 增加一个维度以适配MaxPool1d
        # shape=(32,1,128)

        x = self.maxpool1d2(x)  # shape=(32, 1, 25)
        x = self.batchnorm2(x)
        # out.shape=(32, 1, 25)

        x = x.permute(0, 2, 1)  # 重排维度以适配LSTM输入
        # out.shape=(32, 25, 1)

        # 第二个BiLSTM
        x = self.bilstm2(x)
        # out.shape=(batch=32, 256)

        x = self.dropout(x)
        # x = torch.flatten(x, 1)  # 展平除batch_size外的所有维度, 但是维度已经是(batch, 256)了,没得展了
        x = self.fc(x)
        # x = self.softmax(x)
        return x

In [7]:
# from torchsummary import summary
# net = CNNBiLSTMModel()
# net.to('cpu')  # 确保模型在cpu
# x = torch.rand(32,1,122).to('cpu')
# summary(net,input_size=(1,122), batch_size=32, device='cpu')


In [9]:
# [调试用]打印每一层的输出形状
def print_layer_shapes(model, input_tensor):
    def hook(module, input, output):
        # 这里检查输出是否是元组，如果是的话，我们只打印第一个元素的形状
        if isinstance(output, tuple):
            print(f"{module.__class__.__name__}: {output[0].shape}")
        else:
            print(f"{module.__class__.__name__}: {output.shape}")
    
    # 注册hook
    hooks = []
    for layer in model.children():
        if isinstance(layer, nn.ModuleList):
            for sublayer in layer.children():
                hook_handle = sublayer.register_forward_hook(hook)
                hooks.append(hook_handle)
        else:
            hook_handle = layer.register_forward_hook(hook)
            hooks.append(hook_handle)


    # 前向传播
    with torch.no_grad():
        model(input_tensor)

    # 移除hooks
    for hook in hooks:
        hook.remove()

In [10]:
# [调试用]查看模型每一层的输出
CNN_LSTM_model = CNNBiLSTMModel()
in_tensor = torch.randn(64, 1, 122)  # batch_size=32, in_channels=1, sequence_length=122
print_layer_shapes(CNN_LSTM_model, in_tensor)

Conv1d: torch.Size([64, 64, 122])
MaxPool1d: torch.Size([64, 64, 24])
BatchNorm1d: torch.Size([64, 64, 24])
BiLSTMLayer: torch.Size([64, 128])
MaxPool1d: torch.Size([64, 1, 25])
BatchNorm1d: torch.Size([64, 1, 25])
BiLSTMLayer: torch.Size([64, 256])
Dropout: torch.Size([64, 256])
Linear: torch.Size([64, 5])


In [11]:
# 检查CUDA
def try_device():
    if torch.cuda.is_available():
        # 选择第一个CUDA设备
        device = torch.device("cuda:0")
        print(f"Using CUDA device: {torch.cuda.get_device_name(0)}")
    else:
        device = torch.device("cpu")
        print("CUDA is not available. Using CPU instead.")
    return device

In [12]:
# 定义数据加载器
def loop_data_loder(data_features, data_labels, batch_size):
    # 设置features
    x_columns = data_features.columns  # 取训练features的全部列名,
    x_array = data_features[x_columns].values  # x_array即为本轮循环中,模型的train_features
    # x_array.shape = (-1, 122), x_array.class=ndarray

    # 重塑features.shape为(-1, c_in=1, seq=122),使其符合网络结构输入
    x_features = np.reshape(x_array, (x_array.shape[0], 1, x_array.shape[1]))
    # shape=(-1, 1, 122)

    # 设置Class
    # 如果data_labels已经是一个包含类别名称的Series或者列，你可以这样获取类别索引:
    # 假设data_labels是类别名称的Series，你需要将这些名称映射到索引
    # 首先获取类别名称到索引的映射字典
    label_to_idx = {label: idx for idx, label in enumerate(data_labels.unique())}
    # 然后将类别名称转换为索引
    y_labels = data_labels.replace(label_to_idx).values
    # print(y_labels)
    # print(y_labels.dtype)
    # train_labels = Index(['DOS', 'Probe', 'R2L', 'U2R', 'normal'], dtype='object')
    # train_labels.shape = (-1,5)

    # 创建数据集和数据加载器
    dataset = CustomDataset(x_features, y_labels)
    data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    return data_loader

In [13]:
# 设置随机种子
seed = 42
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# 如果你使用的是cuDNN，还可以设置以下选项来进一步确保确定性
# 确保算法的确定性，即使这可能会牺牲一些性能。
# torch.backends.cudnn.deterministic = True
# 禁用cuDNN的自动性能优化（自动寻找最适合当前配置的卷积算法），因为这些优化可能会导致计算结果的不确定性。
# torch.backends.cudnn.benchmark = False

# 模型参数初始化函数
def init_weights(m):
    if isinstance(m, nn.Conv1d):    # 一维卷积层
        nn.init.xavier_uniform_(m.weight)   # weight用了Xavier均匀初始化
        if m.bias is not None:      # bias存在时，初始化为0
            nn.init.constant_(m.bias, 0)
    elif isinstance(m, nn.LSTM):    # 对于LSTM层
        for name, param in m.named_parameters():
            if 'weight_ih' in name:     # 输入到隐藏层的权重weight_ih
                nn.init.xavier_uniform_(param.data) # 使用了Xavier均匀初始化
                
            elif 'weight_hh' in name:   # 隐藏层到隐藏层的权重weight_hh，
                # 正交初始化通常用于循环神经网络的隐藏层权重，因为它可以帮助维持梯度的规模，从而在训练过程中防止梯度消失或爆炸。
                # todo 测试随机正态分布初始化后的结果是否优于正交初始化
                nn.init.orthogonal_(param.data)     # 正交初始化
                # nn.init.xavier_uniform_(param.data)     # 随机正态分布初始化
                
            elif 'bias' in name:    # 对于LSTM层的偏置项，
                param.data.fill_(0)     #   先所有偏置项初始化为0
                # todo 测试偏置全为0的结果好，还是遗忘门偏置为1的结果好
                n = param.size(0)
                start, end = n // 4, n // 2
                param.data[start:end].fill_(1.) # 然后将遗忘门偏置初始化为1，这在某些情况下可以帮助模型记忆顺序，提高学习能力。
    elif isinstance(m, nn.Linear):  # 线性层使用了Xavier均匀初始化
        nn.init.xavier_uniform_(m.weight)   
        nn.init.constant_(m.bias, 0)    

In [14]:
# 设定超参数
# todo 调参
learning_rate = 0.01
numb_epochs = 30
batch_size = 32
weight_decay = 0.05 # 上一次为0.1，过拟合，train_loss=0.5+, test_loss=2.1+
device = try_device()

Using CUDA device: NVIDIA GeForce RTX 3060 Laptop GPU


In [15]:
# 分层K折
num_folds = 6
k_fold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)  # 随机种子固定,保证每次生成的都一样

In [16]:
k_fold

StratifiedKFold(n_splits=6, random_state=42, shuffle=True)

In [17]:
# 实例化模型
model = CNNBiLSTMModel()

# 初始化模型参数
model.apply(init_weights)

# 模型传入device
print('Training on', device)
model.to(device)

# 设置优化器和损失函数
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, min_lr=1e-5, factor=0.9, patience=5, verbose=True)
loss_fn = nn.CrossEntropyLoss()


Training on cuda:0


In [18]:
# 训练数据加载
train_data = data.copy()    # 不改变源数据
print(data.shape)
print(train_data.shape)
# 下面两项操作都不会改变train_data数据,在模型中不需要改变
labels = train_data['Class']
class_names = list(OrderedDict.fromkeys(labels.to_list())) # 获取列名
print(class_names)
print(f'labels.shape = {labels.shape}')
print(f'labels.dtype = {labels.dtype}')
print(train_data.shape)
features  = train_data.drop(['Class'], axis=1, inplace=False)
print(features.shape)

(148517, 123)
(148517, 123)
['Normal', 'Dos', 'R2L', 'Probe', 'U2R']
labels.shape = (148517,)
labels.dtype = object
(148517, 123)
(148517, 122)


In [19]:
# 初始化存储结构，每个fold的每个epoch的数据都会被存储
fold_metrics = {
    # train
    'train_loss': [[] for _ in range(numb_epochs)],
    'train_accuracy': [[] for _ in range(numb_epochs)], # accuracy两种方法没用区别
    # 加权平均指标
    'train_precision_weighted': [[] for _ in range(numb_epochs)],
    'train_recall_weighted': [[] for _ in range(numb_epochs)],
    'train_f1_weighted': [[] for _ in range(numb_epochs)],
    # 微平均法指标
    'train_precision_micro': [[] for _ in range(numb_epochs)],
    'train_recall_micro': [[] for _ in range(numb_epochs)],
    'train_f1_micro': [[] for _ in range(numb_epochs)],
    
    # val 
    'val_loss': [[] for _ in range(numb_epochs)],
    'val_accuracy': [[] for _ in range(numb_epochs)],   # accuracy两种方法没用区别
    # 加权平均指标
    'val_precision_weighted': [[] for _ in range(numb_epochs)],
    'val_recall_weighted': [[] for _ in range(numb_epochs)],
    'val_f1_weighted': [[] for _ in range(numb_epochs)],
    # 微平均法指标
    'val_precision_micro': [[] for _ in range(numb_epochs)],
    'val_recall_micro': [[] for _ in range(numb_epochs)],
    'val_f1_micro': [[] for _ in range(numb_epochs)],
}
# 初始化用于存储所有混淆矩阵的列表
all_conf_matrices = []

In [20]:
# 绘制混淆矩阵
def plot_confusion_matrix(conf_matrix, class_names, figsize=(10, 8), title='Confusion Matrix'):
    """
    绘制混淆矩阵的函数。

    参数:
    conf_matrix (array-like): 混淆矩阵的数组。
    class_names (list): 类别名称的列表。
    figsize (tuple): 图的大小，默认为(10, 8)。
    title (str): 图的标题，默认为'Confusion Matrix'。
    """
    # class_names = ['Normal', 'Dos', 'R2L', 'Probe', 'U2R']
    plt.figure(figsize=figsize)
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(title)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()  # 调整布局以防止边缘被切割
    plt.show()

# 使用示例：
# 假设combined_conf_matrix是合并后的混淆矩阵，class_names是类别名称列表
# plot_confusion_matrix(combined_conf_matrix, class_names)

In [ ]:
# 完整训练
for epoch in range(numb_epochs):
    start_time = time.time()
    print(f'Epoch {epoch + 1}/{numb_epochs}')
    # 全部K折完算一次epoch, 共需要经历numb_epochs次迭代
    with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True) as prof:
        fold = 0    # fold计数器
        for (train_index, val_index) in k_fold.split(features, labels):
            fold += 1   
            print(f'Fold {fold}/{num_folds}')   # 当前为第fold次K折        
            
            # 根据K折设置训练集和验证集
            train_data, val_data = features.iloc[train_index], features.iloc[val_index]
            train_labels, val_labels = labels.iloc[train_index], labels.iloc[val_index]
            # 只在第一次fold中打印shape
            if fold == 1:
                print(f'train_data.shape = {train_data.shape}, val_data.shape = {val_data.shape}')
                
            # 设置Data_Loder
            train_loader = loop_data_loder(train_data, train_labels, batch_size)
            val_loader = loop_data_loder(val_data, val_labels, batch_size)
            
            # 初始化fold统计数据
            fold_train_loss = 0.0
            fold_train_correct = 0
            fold_train_total = 0
            fold_val_loss = 0.0
            fold_val_correct = 0
            fold_val_total = 0
            fold_train_preds = []
            fold_train_targets = []
            fold_val_preds = []
            fold_val_targets = []
            
            '''在当前fold训练集上训练模型'''
            model.train()
            for train_batch, train_label_batch in train_loader:
                optimizer.zero_grad()
                train_batch, train_label_batch = train_batch.to(device), train_label_batch.to(device)
                
                outputs = model(train_batch)
                loss = loss_fn(outputs, train_label_batch)
                loss.backward()
                optimizer.step()
    
                fold_train_loss += loss.item()
                _, predicted = torch.max(outputs.data, 1)
                fold_train_total += train_label_batch.size(0)
                fold_train_correct += (predicted == train_label_batch).sum().item()
                fold_train_preds.extend(predicted.view(-1).cpu().numpy())
                fold_train_targets.extend(train_label_batch.cpu().numpy())
            
            # 计算训练集上的统计数据
            train_loss = fold_train_loss / len(train_loader)
            train_accuracy = fold_train_correct / fold_train_total
            # 加权平均法计算precision，recall，F1-score
            train_precision_weighted = precision_score(fold_train_targets, fold_train_preds, average='weighted', zero_division=0)
            train_recall_weighted = recall_score(fold_train_targets, fold_train_preds, average='weighted', zero_division=0)
            train_f1_weighted = f1_score(fold_train_targets, fold_train_preds, average='weighted', zero_division=0)
            # 微平均法计算precision，recall，F1-score
            train_precision_micro = precision_score(fold_val_targets, fold_val_preds, average='micro', zero_division=0)
            train_recall_micro = recall_score(fold_val_targets, fold_val_preds, average='micro', zero_division=0)
            train_f1_micro = f1_score(fold_val_targets, fold_val_preds, average='micro', zero_division=0)
            
            '''在当前fold验证集上评估模型'''
            model.eval()
            with torch.no_grad():
                for val_batch, val_label_batch in val_loader:
                    val_batch, val_label_batch = val_batch.to(device), val_label_batch.to(device)
                    outputs = model(val_batch)
                    loss = loss_fn(outputs, val_label_batch)
                    
                    fold_val_loss += loss.item()
                    _, predicted = torch.max(outputs.data, 1)
                    fold_val_total += val_label_batch.size(0)
                    fold_val_correct += (predicted == val_label_batch).sum().item()
                    fold_val_preds.extend(predicted.view(-1).cpu().numpy())
                    fold_val_targets.extend(val_label_batch.cpu().numpy())
            
            '''
            # 在每个fold后更新学习率, 如果你想更频繁地调整学习率，可以在每个fold后调整。
            scheduler.step(fold_val_loss) # 已在全部fold结束后添加
            '''
            
            # 计算并存储当前fold的混淆矩阵
            fold_conf_matrix = confusion_matrix(fold_val_targets, fold_val_preds)
            all_conf_matrices.append(fold_conf_matrix)
            
            # 计算验证集上的统计数据
            '''
            # accuracy，pre，recall，F1-score使用微平均法计算
            # 对于每次的交叉验证，累积每个类别的真正例（TP）、假正例（FP）和假负例（FN）。
            # 使用累积的TP、FP和FN来计算整体的precision、recall和F1分数：
            '''
            val_loss = fold_val_loss / len(val_loader)
            val_accuracy = fold_val_correct / fold_val_total
            # 加权平均法计算precision，recall，F1-score
            val_precision_weighted = precision_score(fold_val_targets, fold_val_preds, average='weighted', zero_division=0)
            val_recall_weighted = recall_score(fold_val_targets, fold_val_preds, average='weighted', zero_division=0)
            val_f1_weighted = f1_score(fold_val_targets, fold_val_preds, average='weighted', zero_division=0)
            # 微平均法计算precision，recall，F1-score
            val_precision_micro = precision_score(fold_val_targets, fold_val_preds, average='micro', zero_division=0)
            val_recall_micro = recall_score(fold_val_targets, fold_val_preds, average='micro', zero_division=0)
            val_f1_micro = f1_score(fold_val_targets, fold_val_preds, average='micro', zero_division=0)
    
            # 将当前fold的统计数据添加到fold_metrics中
            for i in range (1):     # 循环没用，单纯想折叠大段代码
                # train
                fold_metrics['train_loss'][epoch].append(train_loss)
                fold_metrics['train_accuracy'][epoch].append(train_accuracy)
                # 添加加权平均数据
                fold_metrics['train_precision_weighted'][epoch].append(train_precision_weighted)
                fold_metrics['train_recall_weighted'][epoch].append(train_recall_weighted)
                fold_metrics['train_f1_weighted'][epoch].append(train_f1_weighted)
                # 添加微平均法数据
                fold_metrics['train_precision_micro'][epoch].append(train_precision_micro)
                fold_metrics['train_recall_micro'][epoch].append(train_recall_micro)
                fold_metrics['train_f1_micro'][epoch].append(train_f1_micro)
                
                # val
                fold_metrics['val_loss'][epoch].append(val_loss)
                fold_metrics['val_accuracy'][epoch].append(val_accuracy)
                # 加权平均指标
                fold_metrics['val_precision_weighted'][epoch].append(val_precision_weighted)
                fold_metrics['val_recall_weighted'][epoch].append(val_recall_weighted)
                fold_metrics['val_f1_weighted'][epoch].append(val_f1_weighted)
                # 微平均法指标
                fold_metrics['val_precision_micro'][epoch].append(val_precision_micro)
                fold_metrics['val_recall_micro'][epoch].append(val_recall_micro)
                fold_metrics['val_f1_micro'][epoch].append(val_f1_micro)
    
        '''如果你的模型在不同的folds上表现差异很大，使用所有folds的平均损失可能更合适。'''
        # 在所有folds完成后，更新learning_rate
        val_loss_avg = sum(fold_metrics['val_loss'][epoch]) / num_folds
        scheduler.step(val_loss_avg)
        
        # 打印epoch的平均统计数据（如果需要）
        # 循环没用，单纯想折叠大段代码
        for i in range (1):
            print(f'Epoch {epoch + 1} : ')
            print(f'Average Training Loss: {np.mean(fold_metrics["train_loss"][epoch]):.4f}, ')
            print(f'Average Training Accuracy: {np.mean(fold_metrics["train_accuracy"][epoch]):.4f}, ')
            print(f'Average Validation Loss: {np.mean(fold_metrics["val_loss"][epoch]):.4f}, ')
            print(f'Average Validation Accuracy: {np.mean(fold_metrics["val_accuracy"][epoch]):.4f}, ')
    
    print(prof.key_averages().table(sort_by='cpu_time_total', row_limit=10))    
    
    end_time = time.time()
    epoch_time = end_time - start_time
    print(f'Epoch{epoch}/{numb_epochs} took {epoch_time:.2f} seconds')
    print('--------------------------------------\n')

Epoch 1/30
Fold 1/6
train_data.shape = (123764, 122), val_data.shape = (24753, 122)


In [ ]:
# 所有epoch和fold完成后，合并混淆矩阵
combined_conf_matrix = np.sum(all_conf_matrices, axis=0)

# 绘制合并后的混淆矩阵
plot_confusion_matrix(combined_conf_matrix, class_names)

In [ ]:
# val_accuracy 
# 第一张图：每个fold的train_accuracy和模型总体的val_accuracy随epoch变化的曲线
def plot_fold_train_accuracies(fold_metrics, numb_epochs):
    epochs = range(1, numb_epochs + 1)
    plt.figure(figsize=(10, 6))
    
    # for fold in range(num_folds):
    #     plt.plot(epochs, [fold_metrics['train_accuracy'][epoch][fold] for epoch in range(numb_epochs)], label=f'Fold {fold + 1}')
    # 
    # 计算并绘制模型总体的val_accuracy
    overall_train_accuracy = [np.mean([fold_metrics['val_accuracy'][epoch][fold] for fold in range(num_folds)]) for epoch in range(numb_epochs)]
    plt.plot(epochs, overall_train_accuracy, label='Overall Val Accuracy', color='black', linewidth=2, linestyle='--')
    
    plt.title('Val Accuracy per Fold over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Val Accuracy')
    plt.legend()
    plt.show()
    

In [ ]:
# 第二张图：模型总体的train_loss, val_loss随epoch变化的曲线
def plot_overall_metrics(fold_metrics, numb_epochs):
    epochs = range(1, numb_epochs + 1)
    plt.figure(figsize=(10, 6))
    
    # # 绘制模型总体的train_accuracy和val_accuracy
    # overall_train_accuracy = [np.mean(fold_metrics['train_accuracy'][epoch]) for epoch in range(numb_epochs)]
    # overall_val_accuracy = [np.mean(fold_metrics['val_accuracy'][epoch]) for epoch in range(numb_epochs)]
    # plt.plot(epochs, overall_train_accuracy, label='Overall Train Accuracy', marker='o')
    # plt.plot(epochs, overall_val_accuracy, label='Overall Val Accuracy', marker='v')
    
    # 绘制模型总体的train_loss和val_loss
    overall_train_loss = [np.mean(fold_metrics['train_loss'][epoch]) for epoch in range(numb_epochs)]
    overall_val_loss = [np.mean(fold_metrics['val_loss'][epoch]) for epoch in range(numb_epochs)]
    plt.plot(epochs, overall_train_loss, label='Overall Train Loss', marker='s')
    plt.plot(epochs, overall_val_loss, label='Overall Val Loss', marker='*')
    
    plt.title('Overall Training/Validation Metrics over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()
    plt.show()
    

In [ ]:
# 第三张图：模型的accuracy, recall, precision, f1分数随epoch变化的曲线
def plot_performance_metrics(fold_metrics, numb_epochs):
    epochs = range(1, numb_epochs + 1)
    plt.figure(figsize=(10, 6))
    
    # 计算模型总体的performance metrics
    overall_accuracy = [np.mean(fold_metrics['val_accuracy'][epoch]) for epoch in range(numb_epochs)]
    overall_recall_weighted = [np.mean(fold_metrics['val_recall_weighted'][epoch]) for epoch in range(numb_epochs)]
    overall_precision_weighted = [np.mean(fold_metrics['val_precision_weighted'][epoch]) for epoch in range(numb_epochs)]
    overall_f1_weighted = [np.mean(fold_metrics['val_f1_weighted'][epoch]) for epoch in range(numb_epochs)]
    overall_precision_micor = [np.mean(fold_metrics['val_recall_micro'][epoch]) for epoch in range(numb_epochs)]
    overall_recall_micor = [np.mean(fold_metrics['val_recall_micro'][epoch]) for epoch in range(numb_epochs)]
    overall_f1_micor = [np.mean(fold_metrics['val_precision_micro'][epoch]) for epoch in range(numb_epochs)]
    
    
    # 绘制曲线
    plt.plot(epochs, overall_accuracy, label='Accuracy_weighted', marker='x')
    # weighted指标
    plt.plot(epochs, overall_recall_weighted, label='Recall_weighted', marker='v')
    plt.plot(epochs, overall_precision_weighted, label='Precision_weighted', marker='s')
    plt.plot(epochs, overall_f1_weighted, label='F1 Score_weighted', marker='*')
    # micor指标
    plt.plot(epochs, overall_recall_micor, label='Recall_micro', marker='^')
    plt.plot(epochs, overall_precision_micor, label='Precision_micro', marker='p')
    plt.plot(epochs, overall_f1_micor, label='F1_micro', marker='^')
    
    
    plt.title('Performance Metrics over Epochs')
    plt.xlabel('Epochs')
    plt.ylabel('Metrics')
    plt.legend()
    plt.show()

In [ ]:
# 每个fold的train_accuracy和模型总体的train_accuracy随epoch变化的曲线
plot_fold_train_accuracies(fold_metrics, numb_epochs)

In [ ]:
# 模型总体的train_accuracy, train_loss, val_accuracy, val_loss随epoch变化的曲线
plot_overall_metrics(fold_metrics, numb_epochs)

In [ ]:
# 模型的accuracy, recall, precision, f1分数随epoch变化的曲线
plot_performance_metrics(fold_metrics, numb_epochs)